In [1]:
import pandas as pd

In [2]:
exp = pd.read_csv('../data/METABRIC/data_expression.txt',sep='\t')
exp.shape

(24368, 1906)

In [3]:
exp.head(2)

,Hugo_Symbol,Entrez_Gene_Id,MB-0362,MB-0346,MB-0386,MB-0574,MB-0503,MB-0641,MB-0201,MB-0218,...,MB-6122,MB-6192,MB-4820,MB-5527,MB-5167,MB-5465,MB-5453,MB-5471,MB-5127,MB-4313
0,RERE,473.0,8.676978,9.653589,9.033589,8.814855,9.274265,9.286585,8.437347,8.569973,...,8.756024,8.804947,9.991215,9.595923,9.637249,8.131637,9.606915,9.049296,8.858622,8.415867
1,RNF165,494470.0,6.075331,6.687887,5.910885,5.628740,5.908698,6.206729,6.095592,6.383530,...,7.101087,5.601472,7.103160,6.418987,6.203370,9.101942,7.427494,6.850000,6.550450,6.831722


In [7]:
# clean Feature if  missing more 20
(24368 / 100) *  20

4873.6

In [12]:
b = list()
for c in exp.columns:
    missing = exp[c].nonzero()
    if 24368 - len(missing[0])   >= 1:
#        del exp[c]
        print(c)
        

In [5]:
meta_index_exp = exp.columns
meta_index_exp= meta_index_exp[2:]
tops = ['Hugo_Symbol']
tops.extend( list(meta_index_exp))
exp = exp[tops]

In [6]:
exp.shape

(24368, 1905)

In [7]:
(exp.shape[0] /100)* 20

4873.6

In [8]:
for c in exp.columns:
    missing = exp[c].isnull().sum()
    if missing >= 4874:
        del exp[c]
        print(c)

In [9]:
exp.shape

(24368, 1905)

In [10]:
(1904 / 100 )*20

380.79999999999995

In [11]:
#Find is a feature is missing more than 20%
missing = exp.isnull().sum(axis=1).tolist()

todel = list()
for idx, m in enumerate(missing):
    if m >= 381:
        todel.append(idx)
todel

[]

In [58]:
max(exp.isnull().sum(axis=0).tolist())


1

In [59]:
max(exp.isnull().sum(axis=1).tolist())


2

In [27]:
for col in exp:
    a = exp[col].value_counts(dropna=False)
    if a[a.isnull()].shape[0] > 0:
        print(col)

In [35]:
exp[exp['MB-5452'].isnull()]

,Hugo_Symbol,MB-0362,MB-0346,MB-0386,MB-0574,MB-0503,MB-0641,MB-0201,MB-0218,MB-0316,...,MB-6122,MB-6192,MB-4820,MB-5527,MB-5167,MB-5465,MB-5453,MB-5471,MB-5127,MB-4313
11926,MRPL24,8.198578,8.027969,9.030172,9.183349,8.342043,8.005128,8.905652,8.785722,8.640236,...,7.860168,7.881089,8.658549,7.562491,9.163658,7.797186,7.908056,8.45412,8.335357,6.656387


In [36]:
a = exp[exp['MB-5452'].isnull()]

In [45]:
a.isnull().sum(axis=1).get_values()[0]

1

In [53]:
control = list()
for idx, row in exp.iterrows():
    control.append(row.isnull().sum())
max(control)

2

In [12]:
# immputation using KNN 20 neigh
from sklearn import neighbors
import numpy as np

In [13]:
# custom version of  https://github.com/bwanglzu/Imputer.py/blob/master/imputer/imputer.py

class Imputer:
    """Imputer class."""
    def __init():
        return

    def _fit(self, X, column, k=10, is_categorical=False):
        """Fit a knn classifier for missing column.
        - Args:
                X(numpy.ndarray): input data
                column(int): column id to be imputed
                k(int): number of nearest neighbors, default 10
                is_categorical(boolean): is continuous or categorical feature
        - Returns:
                clf: trained k nearest neighbour classifier
        """
        clf = None
        if not is_categorical:
            clf = neighbors.KNeighborsRegressor(n_neighbors=k)
        else:
            clf = neighbors.KNeighborsClassifier(n_neighbors=k)
        # use column not null to train the kNN classifier
        missing_idxes = np.where(pd.isnull(X[:, column]))[0]
        if len(missing_idxes) == 0:
            return None
        X_copy = np.delete(X, missing_idxes, 0)
        X_train = np.delete(X_copy, column, 1)
        # if other columns still have missing values fill with mean
        col_mean = None
        if not is_categorical:
            col_mean = np.nanmean(X, 0)
        else:
            col_mean = np.nanmedian(X, 0)
        for col_id in range(0, len(col_mean) - 1):
            col_missing_idxes = np.where(np.isnan(X_train[:, col_id]))[0]
            if len(col_missing_idxes) == 0:
                continue
            else:
                X_train[col_missing_idxes, col_id] = col_mean[col_id]
        y_train = X_copy[:, column]
        # fit classifier
        clf.fit(X_train, y_train)
        return clf

    def _transform(self, X, column, clf, is_categorical):
        """Impute missing values.
        - Args:
                X(numpy.ndarray): input numpy ndarray
                column(int): index of column to be imputed
                clf: pretrained classifier
                is_categorical(boolean): is continuous or categorical feature
        - Returns:
                X(pandas.dataframe): imputed dataframe
        """
        missing_idxes = np.where(np.isnan(X[:, column]))[0]
        X_test = X[missing_idxes, :]
        X_test = np.delete(X_test, column, 1)
        # if other columns still have missing values fill with mean
        col_mean = None
        if not is_categorical:
            col_mean = np.nanmean(X, 0)
        else:
            col_mean = np.nanmedian(X, 0)
        # fill missing values in each column with current col_mean
        for col_id in range(0, len(col_mean) - 1):
            col_missing_idxes = np.where(np.isnan(X_test[:, col_id]))[0]
            # if no missing values for current column
            if len(col_missing_idxes) == 0:
                continue
            else:
                X_test[col_missing_idxes, col_id] = col_mean[col_id]
        # predict missing values
        y_test = clf.predict(X_test)
        X[missing_idxes, column] = y_test
        return X

    def knn(self, X, column, k=10, is_categorical=False):
        """Impute missing value with knn.
        - Args:
                X(pandas.dataframe): dataframe
                column(str): column name to be imputed
                k(int): number of nearest neighbors, default 10
                is_categorical(boolean): is continuous or categorical feature
        - Returns:
                X_imputed(pandas.dataframe): imputed pandas dataframe
        """
        X, column = self._check_X_y(X, column)
        clf = self._fit(X, column, k, is_categorical)
        if clf is None:
            return X
        else:
            X_imputed = self._transform(X, column, clf, is_categorical)
            return X_imputed

    def _check_X_y(self, X, column):
        """Check input, if pandas.dataframe, transform to numpy array.
        - Args:
                X(ndarray/pandas.dataframe): input instances
                column(str/int): column index or column name
        - Returns:
                X(ndarray): input instances
        """
        column_idx = None
        X = X.select_dtypes(include=[np.number])

        if isinstance(X, pd.core.frame.DataFrame):
            if isinstance(column, str):
                # get index of current column
                column_idx = X.columns.get_loc(column)
            else:
                column_idx = column
            X = X.as_matrix()
        else:
            column_idx = column
        return X, column_idx

In [14]:
imputer = Imputer()

In [15]:
iexp = exp.copy()
#for c in icnv.columns:
#    missing = icnv[c].isnull().sum()
#    if missing > 0:
#        imputed = imputer.knn(icnv, c, k=20 )
#        icnv[c] = imputed[c]
        
#a = imputer.knn(icnv, "MB-5104",k=20)       


In [16]:
for c in iexp.columns:
    missing = iexp[c].isnull().sum()
    if missing > 0:
#        imputed = imputer.knn(icnv, c, k=20 )
        iexp[c] = imputer.knn(iexp, c, k=20,  )
        
#a = imputer.knn(icnv, "MB-5104",k=20)      

In [18]:
iexp.isnull().sum()

Hugo_Symbol    0
MB-0362        0
MB-0346        0
MB-0386        0
MB-0574        0
MB-0503        0
MB-0641        0
MB-0201        0
MB-0218        0
MB-0316        0
MB-0189        0
MB-0891        0
MB-0658        0
MB-0899        0
MB-0605        0
MB-0258        0
MB-0506        0
MB-0420        0
MB-0223        0
MB-0445        0
MB-0199        0
MB-0517        0
MB-0155        0
MB-0428        0
MB-0117        0
MB-0906        0
MB-0249        0
MB-0660        0
MB-0497        0
MB-0434        0
              ..
MB-7089        0
MB-7030        0
MB-7004        0
MB-7119        0
MB-7263        0
MB-7012        0
MB-0174        0
MB-7182        0
MB-7039        0
MB-3797        0
MB-7112        0
MB-0308        0
MB-7230        0
MB-5452        0
MB-6195        0
MB-6317        0
MB-5464        0
MB-5552        0
MB-5547        0
MB-6189        0
MB-6122        0
MB-6192        0
MB-4820        0
MB-5527        0
MB-5167        0
MB-5465        0
MB-5453        0
MB-5471       

In [19]:
iexp.to_csv('METABRIC_Gene_Expression_data_implaced.txt',index=False,sep='\t')

In [20]:
!ls

2018-03-02-preprocessing-metabric-CNV-Cleaning.ipynb
2018-03-02-preprocessing-metabric-EXP-Cleaning.ipynb
METABRICCNV_CNV_data.txt
METABRIC_CNV_data.txt
METABRICCNV_Gene_Expression.txt
METABRICCNV_Survival.txt
METABRIC_Gene_Expression_data_implaced.txt
METABRIC_Gene_Expression.txt
METABRIC_Mirna_Expression.txt
METABRIC_Survival.txt
preprocessing-metabric-CNV.ipynb
preprocessing-metabric.ipynb
Untitled1.ipynb
Untitled2.ipynb
Untitled.ipynb
